In [1]:
import pandas as pd
import os
import re
from ast import literal_eval
import json
# from groupby_toolz import enrich_db
from csv import QUOTE_ALL 

# Params

In [2]:
# fill_na=False                                                      #This is to fill blank values to n/a

In [3]:
customer_name = 'Kimball Midwest'
gdrive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer_name}/_Ready For Upload'

In [4]:
files = os.listdir(gdrive_path)
csv_files = [file for file in files if file[-3:] == 'csv' and file[:7] != 'final -']

In [5]:
df = pd.read_csv(f'{gdrive_path}\\{csv_files[0]}', dtype=str)

In [6]:
df.fillna('n/a',inplace=True)

# import upload files

In [7]:
files = os.listdir(gdrive_path)
csv_files = [file for file in files if file[-3:] == 'csv' and file[:7] != 'final -']

def list_conversion(x):
    try:
        if (type(x) == list) or (str(x).find('[')> -1):
            lst = literal_eval(str(x))
            if len(lst) <= 1:
                return lst[0]
            if len(lst) > 1:
                return str(lst)
        else:
            return str(x)
    except:
        pass

def double_quoted(x,fill_na=True):
    ## loads n/a as nan. reverting them back for proper upload
    if str(x).lower() == 'nan':
        if fill_na == True:
            return r'n/a'
        if fill_na == False:
            return ''
    else:
        return re.sub('\'', '\"', str(x))

for csv_file in csv_files:
    print(f'trying file {csv_file}')
    df = pd.read_csv(f'{gdrive_path}\\{csv_file}', dtype=str)
    curation_columns = [col for col in df.columns if str(col[0].upper())[0:1] == 'Q']
    df['external_id'] = df['external_id'].apply(lambda x: x.zfill(8))
    # processing
    for curation_column in curation_columns:
#         df[curation_column] = df[curation_column].apply(list_conversion)
        df[curation_column] = df[curation_column].apply(lambda x: double_quoted(x,fill_na=fill_na))     # Use for non-na
#         df[curation_column] = df[curation_column].apply(double_quoted)                                    # Use this for N/A values
    # extracting data to plug into enrich db
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset=['external_id'],inplace=True)
    df.to_csv(f'{gdrive_path}\\final - {csv_file}', columns= ['external_id'] + curation_columns,
                                   index=False,
                                   quoting=QUOTE_ALL)

trying file Fasteners - minimum_operating_temperature - na upload.csv
